# Module 3: AI Agents with Semantic Kernel
## Building Intelligent AI Agents

### 1. Introduction to SK Agents
Agents in Semantic Kernel are AI-powered entities that can engage in conversations, make decisions, and execute tasks. They can work independently or collaborate in groups to achieve complex goals.


### 2. Creating Basic Agents


In [ ]:
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents.chat_history import ChatHistory

# Create a kernel and add a chat service
kernel = Kernel()
kernel.add_service(AzureChatCompletion(service_id="agent"))

# Create a simple agent with personality
agent = ChatCompletionAgent(
    service_id="agent",
    kernel=kernel,
    name="Pirate",
    instructions="You are a friendly pirate who always speaks in pirate dialect and ends messages with a parrot sound."
)
kernel = Kernel()
kernel.add_service(AzureChatCompletion(service_id="agent"))


# Create chat history and helper function for interaction
chat = ChatHistory()

async def chat_with_agent(agent: ChatCompletionAgent, message: str):
    """Function to handle agent interaction"""
    chat.add_user_message(message)
    print(f"User: {message}")
    
    # Use streaming for responsive interaction
    chunks = []
    async for chunk in agent.invoke_stream(chat):
        chunks.append(chunk)
        print(chunk.content, end="", flush=True)  # Show response as it comes
    print("\n")  # New line after response
    
    # Add complete response to chat history
    complete_response = "".join([chunk.content for chunk in chunks])
    chat.add_assistant_message(complete_response)

print("Starting chat with Pirate Agent...")
    
    # Test different types of interactions
await chat_with_agent(agent, "Hello! Can you help me find treasure?")
await chat_with_agent(agent, "What's the best way to navigate at sea?")
await chat_with_agent(agent, "Tell me about your parrot!")

In [ ]:
from typing import Annotated
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

class WeatherPlugin:
    """Plugin for weather-related functions"""
    
    @kernel_function(description="Get the current weather for a location.")
    def get_weather(
        self,
        location: Annotated[str, "The city name"]
    ) -> str:
        # In real implementation, this would call a weather API
        return f"The weather in {location} is sunny and 22°C"

    @kernel_function(description="Get the weather forecast for next 3 days.")
    def get_forecast(
        self,
        location: Annotated[str, "The city name"]
    ) -> str:
        return f"3-day forecast for {location}: Sunny, Cloudy, Rain"

# Set up kernel with plugins
kernel = Kernel()
kernel.add_service(AzureChatCompletion(service_id="agent"))

# Configure function auto-invocation
settings = kernel.get_prompt_execution_settings_from_service_id(service_id="agent")
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Add plugin to kernel
kernel.add_plugin(WeatherPlugin(), plugin_name="weather")

# Create agent with access to plugins
agent = ChatCompletionAgent(
    service_id="agent",
    kernel=kernel,
    name="WeatherAssistant",
    instructions="""You help users with weather-related queries.
    Always aim to provide the most accurate and detailed information possible.
    When appropriate, combine current weather with forecast information.""",
    execution_settings=settings
)


chat = ChatHistory()

async def ask_weather(question: str):
    chat.add_user_message(question)
    print(f"User: {question}")

async for response in agent.invoke_stream(chat):
    print(response.content, end="", flush=True)
print("\n")

await ask_weather("What's the weather like in Seattle?")
await ask_weather("Should I pack an umbrella for my trip to London next week?")
await ask_weather("Compare the weather in New York and Tokyo.")

In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.agents.open_ai import AzureAssistantAgent, OpenAIAssistantAgent

class ApprovalTerminationStrategy(TerminationStrategy):
    """
    Termination Strategy for Peer Review Discussion.

    This strategy evaluates the chat history to determine if the discussion
    should terminate based on achieving consensus or a clear approval.
    """

    async def should_agent_terminate(self, agent, history):
        """Evaluate termination condition."""
        return "approved" in history[-1].content.lower()


# Configuration for agents' personas and behavior
RESEARCHER_NAME = "AIResearcher"
RESEARCHER_INSTRUCTIONS = """
Role: AI Researcher
Objective: Refine and explain the technical details of a proposed model.
Actions:
- Answer questions concisely while focusing on scientific rigor.
- Propose refinements to improve the clarity and reproducibility of the paper.
- Avoid discussing unrelated topics; stay focused on the research at hand.
"""

REVIEWER_NAME = "PeerReviewer"
REVIEWER_INSTRUCTIONS = """
Role: Peer Reviewer
Objective: Evaluate the proposed research paper for clarity, significance, and rigor.
Actions:
- Identify ambiguities or potential improvements in methodology or claims.
- Approve the paper if it meets standards, using the word 'approved'.
- Suggest actionable refinements without rephrasing the entire paper.
"""


class ResearchToolsPlugin:
    """
    Plugin for Providing Research-Related Context and Tools.

    This plugin supports the agents by offering functionality for citing references,
    summarizing research papers, and validating datasets.
    """

    @kernel_function(description="Provides a formatted citation for a given paper.")
    def cite_paper(
        self, title: Annotated[str, "The title of the paper."],
        author: Annotated[str, "The author of the paper."],
        year: Annotated[int, "The year of publication."]
    ) -> Annotated[str, "Returns the citation in APA format."]:
        """Generate a citation."""
        return f"{author} ({year}). {title}. Journal of AI Research."

    @kernel_function(description="Summarizes the key contributions of a paper.")
    def summarize_paper(
        self, abstract: Annotated[str, "The abstract of the paper."]
    ) -> Annotated[str, "Returns a concise summary of the paper's contributions."]:
        """Summarize a research paper's contributions."""
        return f"Key Contributions: {abstract[:200]}..."  # Truncated for brevity.


def initialize_kernel_with_research_tools(service_id: str) -> Kernel:
    """
    Initialize a Semantic Kernel instance with Azure Chat Completion and research tools.

    Args:
        service_id (str): Identifier for the chat service.

    Returns:
        Kernel: Configured Semantic Kernel instance.
    """
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id))
    kernel.add_plugin(plugin=ResearchToolsPlugin(), plugin_name="research_tools")
    return kernel


# Initialize the Kernel and agents
kernel = initialize_kernel_with_research_tools("peerreviewer")
settings = kernel.get_prompt_execution_settings_from_service_id(service_id="peerreviewer")
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

agent_researcher = ChatCompletionAgent(
    service_id="peerreviewer",
    kernel=kernel,
    name=RESEARCHER_NAME,
    instructions=RESEARCHER_INSTRUCTIONS,
    execution_settings=settings,
)

agent_reviewer = ChatCompletionAgent(
    service_id="peerreviewer",
    kernel=kernel,
    name=REVIEWER_NAME,
    instructions=REVIEWER_INSTRUCTIONS,
    execution_settings=settings,
)



# Define group chat with termination strategy
chat = AgentGroupChat(
    agents=[agent_researcher, agent_reviewer],
    termination_strategy=ApprovalTerminationStrategy(
        agents=[agent_reviewer], maximum_iterations=10
    ),
)

# User initiates the conversation
user_input = "Refine the methodology section of the proposed AI model paper."
await chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))
print(f"# {AuthorRole.USER}: '{user_input}'")

async for content in chat.invoke():
            print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

print(f"# Chat Complete: {chat.is_complete}")


In [ ]:
# Persönliches Agentenframewok Schwarm


from schwarm.core.schwarm import Schwarm
from schwarm.models.types import Agent
from schwarm.provider.litellm_provider import LiteLLMConfig

# Create an agent with the name "hello_agent" and the default provider configurations
hello_agent = Agent(name="hello_agent", configs=[LiteLLMConfig()])

# Start the agent
Schwarm(application_mode='server').quickstart(agent=hello_agent)